<font color=gray>Oracle Cloud Infrastructure Data Science Sample Notebook

Copyright (c) 2021 Oracle, Inc.  All rights reserved. <br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>

# Uploading Larger Size Model Artifact Using OCI Pyhton SDK 

This notebook demonstrates simple solution for OCI Python SDK which allows data scientists to upload larger model artifacts and eliminate the timeout error that is experienced by most folks when the artifact is large. It shows end-to-end steps from setting up the configuration till uploading the model artifact.

## Pre-requisites to Running this Notebook 

* We recommend that you run this notebook in a notebook session using the **Data Science Conda Environment "Data Exploration and Manipulation for CPU Python 3.7 V2 conda environment"** 
* You need access to the public internet
* Upgrade the current version of the OCI Python SDK (`oci`): 

In [ ]:
!pip install --upgrade oci

In [ ]:
import oci
from oci.data_science import DataScienceClient
from oci.data_science.models import CreateModelDetails, CreateProjectDetails, \
    CreateModelProvenanceDetails, Project, Model, UpdateModelDetails
import os
import logging

REGION = f"<replace-with-region>"
logger = logging.getLogger('upload_model_artifact')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

def fetch_config() -> dict:
    """
    Gets the oci config stored locally.

    Returns:
    config (dict): a config object
    """

    try:
        config = oci.config.from_file()
        config['region'] = REGION

        # We can see what our config file looks like by using 'print(config)'.
        return config
    except Exception as e:
        logger.error("Failed while getting oci config : %s", format(e))
        exit(1)
        
config = fetch_config()

In [ ]:
SERVICE_ENDPOINT = f"<replace-with-service-endpoint>"

def create_data_science_client(config: dict) -> DataScienceClient:
    """
    Creates a data science client using the config.

    Parameters:
    config (dict): the oci config

    Returns:
    data_science (DataScienceClient): a data science client object
    """

    logger.info("Setting up data science client and overriding service endpoint...")
    try:
        data_science = DataScienceClient(config, service_endpoint=SERVICE_ENDPOINT)
        logger.info("Successfully set up data science client!")
        return data_science
    except Exception as e:
        logger.error("Failed to set up data science client with error: %s", format(e))
        exit(1)
        
data_science_client = create_data_science_client(config)

In [ ]:
# set Compartment Id
COMPARTMENT_ID = os.environ['NB_SESSION_COMPARTMENT_OCID']
PROJECT_DESCRIPTION = f"<replace-with-your-project-description>"
PROJECT_DISPLAY_NAME = f"<replace-with-your-project-display-name>"

data_science_models = oci.data_science.models

def create_project_details(data_science_models) -> CreateProjectDetails:
    """
    Creates the project details.

    Parameters:
    data_science_models (module): the data science models to import create project details model from

    Returns:
    create_project_details_object (CreateProjectDetails): a create project details model object
    """

    logger.info("Defining project details object...")
    
    # We need to create a project first. Get the create project details object.
    create_project_details_object = data_science_models.CreateProjectDetails()

    # Set the project compartment id.
    create_project_details_object.compartment_id = COMPARTMENT_ID

    # Set the project display name.
    create_project_details_object.display_name = PROJECT_DISPLAY_NAME

    # Set the project description
    create_project_details_object.description = PROJECT_DESCRIPTION
    # logger.info("Project details: %s", create_project_details_object)
    return create_project_details_object
create_project_details_object = create_project_details(data_science_models)

In [ ]:
def create_project(data_science_client: DataScienceClient, create_project_details_object: CreateProjectDetails) -> str:
    """
    Creates the project.

    Parameters:
    data_science_client (DataScienceClient): the data science client
    create_project_details_object (CreateProjectDetails): the create project details model object

    Returns:
    project_id (str): a project ocid
    """

    try:
        logger.info("Creating project...")
        # print(data_science_client.base_client.__dict__)
        project = data_science_client.create_project(create_project_details_object)
        # logger.info("Data returned: %s", project.data)
        # logger.info("Grabbing the project ocid...")
        project_id = json.loads(str(project.data))['id']
        # logger.info("Project ocid: %s", project_id)
        return project_id
    except Exception as e:
        logger.error("Failed to create project with error: %s", format(e))
        exit(1)

In [ ]:
def create_project_main():
    project_id = create_project(data_science_client, create_project_details_object)
    logger.info(project_id)
    return str(project_id)

project_id = create_project_main()

In [ ]:
def create_model_details(data_science_models, project_id: str):
    """
    Creates the model details object

    Parameters:
    data_science_models (module): the data science models to import create model details model from
    project_id (str): the project ocid

    Returns:
    create_model_details_object (CreateModelDetails): a create model details object
    """
    logger.info("Defining Model details object...")
    create_model_details_object = data_science_models.CreateModelDetails()
    create_model_details_object.compartment_id = COMPARTMENT_ID
    create_model_details_object.display_name = f"<replace-with-your-object-display-name>"
    create_model_details_object.project_id = project_id
    return create_model_details_object

def create_model(data_science_client: DataScienceClient, create_model_details_object: CreateModelDetails):
    """
    Creates the model.

    Parameters:
    data_science_client (DataScienceClient): the data science client
    create_model_details_object (CreateModelDetails): a create model details object to use in creating models

    Returns:
    model_id (str): the model ocid generated.
    """
    try:
        logger.info("Creating model...")
        model = data_science_client.create_model(
            create_model_details_object)
        logger.info(f'model details: {model}')
        model_id = json.loads(str(model.data))['id']
        return model_id
    except Exception as e:
        logger.error("Failed to create model with error: %s", format(e))
        exit(1)

In [ ]:
def create_model_main(project_id: str):
    # def create_model_main():
    logger.info("creating model...")
    # creates the model and model artifact
    # Make sure to provide the correct path of the zip file in ARTIFACT_FILE_NAME
    create_model_details_object = create_model_details(data_science_models, project_id)
    model_id = create_model(data_science_client, create_model_details_object)
    # create_model_artifact(data_science_client, model_id)
    logger.info(model_id)
    return str(model_id)

model_id = create_model_main(create_project_main())

In [ ]:
# provide the artifact file path
ARTIFACT_FILE_NAME = f"<replace-with-your-artifact-file-path>"

def upload_model_artifact(model_id: str):
    try:
        logger.info("uploading model artifact...")
        # creates the model and model artifact
        # Make sure to provide the correct path of the zip file in ARTIFACT_FILE_NAME
        create_model_artifact(data_science_client, model_id)
    except Exception as e:
        return str(e)
    return "True"

def create_model_artifact(data_science_client: DataScienceClient, model_id: str):
    """
    Creates the model artifact.

    Parameters:
    data_science_client (DataScienceClient): the data science client
    model_id (str): the model id to use in creating artifact
    """
    logger.info("Create artifact")
    f = open(ARTIFACT_FILE_NAME, "rb")
    logger.info("File open")
    content_disposition = "attachment;filename={}".format(ARTIFACT_FILE_NAME)
    logger.debug(content_disposition)
    logger.debug(data_science_client)
    try:
        data_science_client.base_client.timeout = 30 * 60
        data_science_client.create_model_artifact(
            model_id, f, content_disposition=content_disposition)
        logger.info("Upload success")
        print("Upload Success")
        logger.info("Finished creating artifact")
    except Exception as e:
        print("==================", str(e))
        logger.error("Upload error")
        logger.debug(str(e))
    f.close()
    return